In [ ]:
%load_ext memory_profiler
%memit
import pandas as pd
import numpy as np


data = np.zeros((100000, 5))
data = pd.DataFrame(data, columns=['col%d'%i for i in range(data.shape[1])])
data.to_parquet('tmp.parquet')

data = np.zeros((100000, 5))
data = pd.DataFrame(data, columns=['col%d'%i for i in range(data.shape[1])])
data.to_csv('tmp.csv', index=False)

In [ ]:
%memit
from dask import dataframe as DF

df = DF.read_parquet('tmp.parquet')
df = df.repartition(partition_size="100KB")
# part_path = ['tmpcsv/part.{}'.format(i) for i in range(df.npartitions)]

def name_fn(i):
    return 'mypart.{}'.format(i)

df.to_parquet(path='tmpparquet', write_metadata_file=False, name_function=name_fn)

In [ ]:
data = DF.read_parquet('./tmpp/part.0.parquet')
data = data.compute()
import sys
sys.getsizeof(data)
print(data.shape)
data

In [ ]:
%memit
from dask import dataframe as DF

df = DF.read_csv('tmp.csv')
df = df.repartition(partition_size="100KB")
part_path = ['tmpcsv/part.{}'.format(i) for i in range(df.npartitions)]
df.to_csv(part_path, index=False)

In [ ]:
%memit
import pickle

for part in df.partitions:
    tmp = pickle.dumps(part.compute())
    print(tmp)
    break

In [24]:
import logging
import boto3
from botocore.exceptions import ClientError

import pickle

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [25]:
bucket = 'vuzhuangwei'
create_bucket(bucket)

True

In [26]:
client = boto3.client('s3')
key = 'README.md'
with open(key, 'r') as f:
    val = f.readlines()
    val = pickle.dumps(val)
client.put_object(Bucket=bucket, Key=key, Body=val)

{'ResponseMetadata': {'RequestId': 'M8HT4JWTRB2JB45W',
  'HostId': 'cdKfim5p/ws6ySQHqc9tbZnYnzjWwa8GoyBCol86a9SfVDdktsB8ZmMp8GQTzFuuBqHWMK4BIsU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cdKfim5p/ws6ySQHqc9tbZnYnzjWwa8GoyBCol86a9SfVDdktsB8ZmMp8GQTzFuuBqHWMK4BIsU=',
   'x-amz-request-id': 'M8HT4JWTRB2JB45W',
   'date': 'Fri, 12 Aug 2022 18:48:33 GMT',
   'etag': '"9c0265f933cf1a0651f15854d2009a45"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9c0265f933cf1a0651f15854d2009a45"'}

In [30]:
meta = client.get_object(Bucket=bucket, Key=key)
data = meta['Body'].read()
data

b'\x80\x04\x95\x14\x00\x00\x00\x00\x00\x00\x00]\x94\x8c\r# FS-KV-Store\x94a.'